In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
from sklearn.cluster import KMeans
import json
from pandas.io.json import json_normalize 
from bs4 import BeautifulSoup
import lxml
!pip install beautifulsoup4
import lxml.html as lh
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

# Neighborhoods coordinates

I will use the data from the police department

In [2]:
# Get the data from the police department of San Francisco
link_policesf=('https://data.sfgov.org/api/views/wg3w-h783/rows.csv?accessType=DOWNLOAD')
data = pd.read_csv(link_policesf)

In [3]:
data.shape

(196263, 34)

In [4]:
df_incisf=data.dropna(subset = ['Latitude','Longitude'])

In [5]:
limit = 300
df_incidents = df_incisf.iloc[0:limit, :]

In [6]:
# transform the name of the columns to something easy to call
cols = df_incidents.columns
df_incisf1=df_incidents
cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, (str, bytes)) else x)
df_incisf1.columns = cols

In [7]:
# create a new table only with the information I need
df_incisf2=df_incisf1[['Analysis_Neighborhood','Latitude','Longitude']]

In [8]:
coorde_sf=df_incisf2.groupby(['Analysis_Neighborhood']).mean()

In [9]:
coord_sf=pd.DataFrame(coorde_sf).reset_index()

In [10]:
coord_sf.shape

(35, 3)

In [11]:
coord_sf.head()

,Analysis_Neighborhood,Latitude,Longitude
0,Bayview Hunters Point,37.731008,-122.391071
1,Bernal Heights,37.740302,-122.418283
2,Castro/Upper Market,37.762716,-122.432138
3,Chinatown,37.797269,-122.407470
4,Excelsior,37.720399,-122.429003


# Retrieve Foursquare data

In [12]:
# now that we have locations for each neighborhood in San Francisco we can retrieve foursquare information
# Foursquare id
CLIENT_ID = 'ZPHNQ3I3HJEOFHFMEWTVAOCCCNHKYFIA0OFY2CULH20ED52O'
CLIENT_SECRET = 'HFS0XTAD4SW50GMAYPETN4VS15AICEVABN4UDBV5QW1ZZRSH' 
VERSION = '20180605'

In [13]:
# save the geolocation for san francisco
address = 'San Francisco'

geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [14]:
# check for the first neighborhood in the dataframe
neighborhood_sf_latitude = coord_sf.loc[0, 'Latitude'] 
neighborhood_sf_longitude = coord_sf.loc[0, 'Longitude'] 

neighborhood_name = coord_sf.loc[0, 'Analysis_Neighborhood'] # neighborhood name
#print the values that are going to be used for the first neighborhood
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_sf_latitude, 
                                                               neighborhood_sf_longitude))

Latitude and longitude values of Bayview Hunters Point are 37.73100818002972, -122.39107098994621.


In [15]:
LIMIT = 200
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_sf_latitude, 
    neighborhood_sf_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=ZPHNQ3I3HJEOFHFMEWTVAOCCCNHKYFIA0OFY2CULH20ED52O&client_secret=HFS0XTAD4SW50GMAYPETN4VS15AICEVABN4UDBV5QW1ZZRSH&v=20180605&ll=37.73100818002972,-122.39107098994621&radius=500&limit=200'

In [16]:
results = requests.get(url).json()

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Bayview Hunters Point YMCA,Gym,37.731851,-122.389733
1,Radio Africa & Kitchen,African Restaurant,37.734826,-122.390764
2,Curio Parlor,Piercing Parlor,37.728394,-122.395273
3,JJ Fish & Chicken,Southern / Soul Food Restaurant,37.732278,-122.391780
4,Craftsman and Wolves Den,Bakery,37.726908,-122.391536


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
# get venues from all neighborhoods
sf_venues = getNearbyVenues(names=coord_sf['Analysis_Neighborhood'],
                                   latitudes=coord_sf['Latitude'],
                                   longitudes=coord_sf['Longitude']
                                  )

Bayview Hunters Point
Bernal Heights
Castro/Upper Market
Chinatown
Excelsior
Financial District/South Beach
Golden Gate Park
Haight Ashbury
Hayes Valley
Inner Richmond
Inner Sunset
Japantown
Lakeshore
Lone Mountain/USF
Marina
Mission
Mission Bay
Nob Hill
Noe Valley
North Beach
Oceanview/Merced/Ingleside
Outer Mission
Outer Richmond
Pacific Heights
Portola
Potrero Hill
Russian Hill
Seacliff
South of Market
Sunset/Parkside
Tenderloin
Twin Peaks
Visitacion Valley
West of Twin Peaks
Western Addition


In [21]:
sf_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bayview Hunters Point,37.731008,-122.391071,Bayview Hunters Point YMCA,37.731851,-122.389733,Gym
1,Bayview Hunters Point,37.731008,-122.391071,Radio Africa & Kitchen,37.734826,-122.390764,African Restaurant
2,Bayview Hunters Point,37.731008,-122.391071,Curio Parlor,37.728394,-122.395273,Piercing Parlor
3,Bayview Hunters Point,37.731008,-122.391071,JJ Fish & Chicken,37.732278,-122.391780,Southern / Soul Food Restaurant
4,Bayview Hunters Point,37.731008,-122.391071,Craftsman and Wolves Den,37.726908,-122.391536,Bakery


In [22]:
sf_venues_total=sf_venues.groupby(['Neighborhood']).count().reset_index()

In [23]:
print('There are {} uniques categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 300 uniques categories.


# Create a table with only the proportion of food related queries

In [24]:
# food related categories from Venues Categories
food_related= ['African Restaurant', 'Southern / Soul Food Restaurant', 'Mexican Restaurant', 'Chinese Restaurant',\
               'Café', 'BBQ Joint', 'Bakery', 'Restaurant', 'Convenience Store', 'Sushi Restaurant', \
               'American Restaurant', 'Deli / Bodega', 'Food', 'Japanese Restaurant', 'Gay Bar','Breakfast Spot', \
               'Sandwich Place', 'Asian Restaurant', 'New American Restaurant', 'Coffee Shop', 'Cocktail Bar',\
               'Dive Bar', 'Brewery', 'Dessert Shop', 'Burger Joint', 'Mediterranean Restaurant', 'Seafood Restaurant', \
               'Tiki Bar', 'Spanish Restaurant', 'Candy Store', 'Juice Bar', 'Scandinavian Restaurant', 'Vietnamese Restaurant',\
               'Latin American Restaurant', 'French Restaurant', 'Ice Cream Shop', 'Pizza Place', 'Korean Restaurant',\
               'Ramen Restaurant', 'Wine Shop', 'Bar', 'Indian Restaurant', 'Middle Eastern Restaurant', 'Thai Restaurant',\
               'Szechuan Restaurant', 'Wine Bar', 'Italian Restaurant', 'Dim Sum Restaurant', 'Karaoke Bar', 'Tea Room',\
               'Shanghai Restaurant', 'Diner', 'Jiangsu Restaurant', 'Bubble Tea Shop', 'Vegetarian / Vegan Restaurant',\
               'Tuscan Restaurant', 'Argentinian Restaurant', 'Gastropub', 'Hunan Restaurant', 'Filipino Restaurant',\
               'Liquor Store', 'Fast Food Restaurant', 'Creperie', 'Moroccan Restaurant', 'Fried Chicken Joint', 'Salad Place',\
               'Greek Restaurant', 'Chocolate Shop', 'Wagashi Place', 'Food Truck', 'Street Food Gathering', 'Taco Place',\
               'Food Stand', 'Cupcake Shop', 'Caribbean Restaurant', 'Tapas Restaurant', 'Cheese Shop', \
               'Donut Shop', 'German Restaurant', 'Beer Garden', 'Bagel Shop', 'Frozen Yogurt Shop', 'Bistro',\
               'Food & Drink Shop', 'Burmese Restaurant', 'Turkish Restaurant', 'Beer Bar', 'Pub', 'Ethiopian Restaurant',\
               'Nabe Restaurant', 'Peruvian Restaurant', 'Irish Pub', 'Falafel Restaurant', 'Lounge',\
               'Udon Restaurant', 'Snack Place', 'Noodle House', 'Hotpot Restaurant', 'Mac & Cheese Joint', 'Burrito Place', \
               'Cajun / Creole Restaurant', 'South American Restaurant', 'Salvadoran Restaurant', 'Hot Dog Joint',\
               'Cuban Restaurant', 'Hawaiian Restaurant', 'Afghan Restaurant', 'Portuguese Restaurant', 'Kebab Restaurant',\
               'Brazilian Restaurant', 'Sports Bar', 'Beer Store', 'Fondue Restaurant', 'Hotel Bar', 'Dumpling Restaurant',\
               'Pakistani Restaurant']

In [25]:
sf_venues.shape

(2045, 7)

In [26]:
# Get the index of the rows with a food related category
index_food = sf_venues[ sf_venues['Venue Category'].isin(food_related) ].index
index_food.shape

(1211,)

In [27]:
sf_venues_food=sf_venues.iloc[index_food,:]
#sf_venues_food = sf_venues.drop(bus_sf2[(bus_sf2.new == 'diff')].index)

In [28]:
sf_venues_food.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2034,Western Addition,37.783699,-122.426694,Festa,37.785206,-122.431575,Karaoke Bar
2036,Western Addition,37.783699,-122.426694,Jitlada Thai Restaurant,37.786875,-122.430101,Thai Restaurant
2040,Western Addition,37.783699,-122.426694,Arang Restaurant,37.783402,-122.432311,Korean Restaurant
2041,Western Addition,37.783699,-122.426694,College Market,37.783605,-122.422558,Deli / Bodega
2044,Western Addition,37.783699,-122.426694,Burger King,37.783137,-122.421214,Fast Food Restaurant


In [29]:
sf_venues_food.shape

(1211, 7)

In [30]:
sf_ven=sf_venues_food.groupby(['Neighborhood']).count()

In [31]:
sf_ven=pd.DataFrame(sf_ven).reset_index()

In [32]:
sf_ven.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bayview Hunters Point,10,10,10,10,10,10
1,Bernal Heights,37,37,37,37,37,37
2,Castro/Upper Market,61,61,61,61,61,61
3,Chinatown,83,83,83,83,83,83
4,Excelsior,2,2,2,2,2,2


In [33]:
sf_stats=pd.merge(sf_venues_total, sf_ven[['Neighborhood', 'Venue']], on='Neighborhood', how='left')

In [34]:
sf_stats.fillna(0,inplace=True)

In [35]:
sf_stats['Venue']=sf_stats['Venue_y']/sf_stats['Venue_x']

In [36]:
sf_stats.drop(['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue_x', 'Venue Latitude', 'Venue Longitude', 'Venue Category', 'Venue_y'],axis=1, inplace=True)

In [37]:
sf_stats.head()

,Neighborhood,Venue
0,Bayview Hunters Point,0.500000
1,Bernal Heights,0.649123
2,Castro/Upper Market,0.610000
3,Chinatown,0.830000
4,Excelsior,0.333333


# Rent Dataset of san Francisco by neighborhood

In [38]:
sf_rent=pd.read_csv('sf_rent.csv',sep=';')
sf_rent.head()

,Neighborhood,Average Rent
0,Treasure Island,2616
1,Van Ness - Civic Center,2905
2,Tenderloin,2905
3,Marina,2919
4,Downtown District 8 - North East,2931


In [39]:
sf_rent.rename(columns={'Average Rent':'Avg_Rent'}, inplace=True)

# Incidents table

In [40]:
data.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,point,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,HSOC Zones as of 2018-06-05,OWED Public Spaces,Central Market/Tenderloin Boundary Polygon - Updated,Parks Alliance CPSI (27+TL sites)
0,2018/12/02 12:45:00 AM,2018/12/02,00:45,2018,Sunday,2018/12/02 01:56:00 AM,74374327130,743743,180908554,183360210.0,...,-122.404795,"(37.78490829943, -122.40479506276)",32.0,5.0,10.0,8.0,NaN,NaN,NaN,NaN
1,2018/12/01 08:30:00 PM,2018/12/01,20:30,2018,Saturday,2018/12/01 09:18:00 PM,74370071000,743700,180908112,183353564.0,...,-122.408036,"(37.786409612811, -122.408036237445)",19.0,6.0,3.0,36.0,NaN,NaN,1.0,NaN
2,2019/03/18 02:01:00 PM,2019/03/18,14:01,2019,Monday,2019/03/18 02:21:00 PM,78164004134,781640,190194129,190772267.0,...,-122.406699,"(37.756833733806, -122.406699002688)",53.0,3.0,2.0,20.0,3.0,NaN,NaN,NaN
3,2019/03/20 08:00:00 AM,2019/03/20,08:00,2019,Wednesday,2019/03/20 02:06:00 PM,78169706244,781697,190199583,190792201.0,...,-122.404865,"(37.78400661242, -122.404864795177)",32.0,1.0,10.0,34.0,NaN,NaN,NaN,NaN
4,2019/03/12 01:30:00 PM,2019/03/12,13:30,2019,Tuesday,2019/03/15 06:02:00 PM,78154706372,781547,196055103,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# transform the name of the columns to something easy to call
colsin = data.columns
df_i=data
colsin = colsin.map(lambda x: x.replace(' ', '_') if isinstance(x, (str, bytes)) else x)
df_i.columns = colsin

In [42]:
df_i.head()

,Incident_Datetime,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Report_Datetime,Row_ID,Incident_ID,Incident_Number,CAD_Number,...,Longitude,point,SF_Find_Neighborhoods,Current_Police_Districts,Current_Supervisor_Districts,Analysis_Neighborhoods,HSOC_Zones_as_of_2018-06-05,OWED_Public_Spaces,Central_Market/Tenderloin_Boundary_Polygon_-_Updated,Parks_Alliance_CPSI_(27+TL_sites)
0,2018/12/02 12:45:00 AM,2018/12/02,00:45,2018,Sunday,2018/12/02 01:56:00 AM,74374327130,743743,180908554,183360210.0,...,-122.404795,"(37.78490829943, -122.40479506276)",32.0,5.0,10.0,8.0,NaN,NaN,NaN,NaN
1,2018/12/01 08:30:00 PM,2018/12/01,20:30,2018,Saturday,2018/12/01 09:18:00 PM,74370071000,743700,180908112,183353564.0,...,-122.408036,"(37.786409612811, -122.408036237445)",19.0,6.0,3.0,36.0,NaN,NaN,1.0,NaN
2,2019/03/18 02:01:00 PM,2019/03/18,14:01,2019,Monday,2019/03/18 02:21:00 PM,78164004134,781640,190194129,190772267.0,...,-122.406699,"(37.756833733806, -122.406699002688)",53.0,3.0,2.0,20.0,3.0,NaN,NaN,NaN
3,2019/03/20 08:00:00 AM,2019/03/20,08:00,2019,Wednesday,2019/03/20 02:06:00 PM,78169706244,781697,190199583,190792201.0,...,-122.404865,"(37.78400661242, -122.404864795177)",32.0,1.0,10.0,34.0,NaN,NaN,NaN,NaN
4,2019/03/12 01:30:00 PM,2019/03/12,13:30,2019,Tuesday,2019/03/15 06:02:00 PM,78154706372,781547,196055103,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df_i['Incident_Date']=pd.to_datetime(df_i['Incident_Date'])

In [44]:
df_i1=df_i.groupby(['Analysis_Neighborhood', pd.Grouper(key='Incident_Date', freq='M')]).count()

In [45]:
df_i1.reset_index(inplace=True)

In [46]:
df_i1.drop(['Incident_Time', 'Incident_Year', 'Incident_Day_of_Week',
       'Report_Datetime', 'Row_ID', 'Incident_ID', 'Incident_Number',
       'CAD_Number', 'Report_Type_Code', 'Report_Type_Description',
       'Filed_Online', 'Incident_Code', 'Incident_Category',
       'Incident_Subcategory', 'Incident_Description', 'Resolution',
       'Intersection', 'CNN', 'Police_District', 'Supervisor_District',
       'Latitude', 'Longitude', 'point', 'SF_Find_Neighborhoods',
       'Current_Police_Districts', 'Current_Supervisor_Districts',
       'Analysis_Neighborhoods', 'HSOC_Zones_as_of_2018-06-05',
       'OWED_Public_Spaces',
       'Central_Market/Tenderloin_Boundary_Polygon_-_Updated',
       'Parks_Alliance_CPSI_(27+TL_sites)'], axis=1, inplace=True)

In [47]:
df_i1.rename(columns={'Analysis_Neighborhood': 'Neighborhood', 'Incident_Date': 'Date', 'Incident_Datetime': 'Incidents'}, inplace=True)

In [48]:
df_i1.head()

,Neighborhood,Date,Incidents
0,Bayview Hunters Point,2018-01-31,765
1,Bayview Hunters Point,2018-02-28,685
2,Bayview Hunters Point,2018-03-31,631
3,Bayview Hunters Point,2018-04-30,703
4,Bayview Hunters Point,2018-05-31,695


In [49]:
df_i1.shape

(656, 3)

In [50]:
df_i1.describe()

,Incidents
count,656.000000
mean,282.422256
std,329.913988
min,2.000000
25%,109.750000
50%,175.000000
75%,295.000000
max,1655.000000


# Business table of open and closed restaurants

In [51]:
businesses_link= ('https://data.sfgov.org/api/views/g8m3-pdis/rows.csv?accessType=DOWNLOAD')

In [52]:
business_sf=pd.read_csv(businesses_link)
business_sf.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Location Id,Business Account Number,Ownership Name,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,...,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_bh8s_q3mv,:@computed_region_yftq_j783,:@computed_region_rxqg_mtj9,:@computed_region_jx4q_fizf,:@computed_region_fyvs_ahh9,:@computed_region_p5aj_wyqh
0,0000024-02-999,24,Eastman Kodak Co,Eastman Kodak Co,343 State St,Rochester,NY,14650.0,01/01/2003,NaN,...,NaN,NaN,NaN,NaN,26991.0,NaN,NaN,NaN,NaN,NaN
1,0000028-02-001,28,Ferrando Louise,3101 Laguna Apts,3101 Laguna St,San Francisco,CA,94123.0,09/30/1993,NaN,...,15.0,4.0,6.0,13.0,57.0,13.0,1.0,6.0,17.0,9.0
2,0000052-01-001,52,Ideal Novak Corp,Ideal Novak Corp,8 Mendosa Ave,San Francisco,CA,94116.0,10/01/1968,NaN,...,45.0,10.0,8.0,41.0,29491.0,1.0,4.0,5.0,40.0,8.0
3,0000071-01-001,71,Tournahu George L,Tournahu Arms,1842 Jefferson St,San Francisco,CA,94123.0,10/01/1968,NaN,...,17.0,4.0,6.0,13.0,57.0,13.0,1.0,6.0,17.0,9.0
4,0000071-02-001,71,Tournahu George L,3301 Broderick Apartments,3301 Broderick St,San Francisco,CA,94123.0,10/01/1968,NaN,...,17.0,4.0,6.0,13.0,57.0,13.0,1.0,6.0,17.0,9.0


In [53]:
business_sf.shape

(236905, 36)

In [54]:
# transform the name of the columns to something easy to call
colsb = business_sf.columns
df_bus=business_sf
colsb = colsb.map(lambda x: x.replace(' ', '_') if isinstance(x, (str, bytes)) else x)
df_bus.columns = colsb

In [55]:
df_bus.NAICS_Code_Description.unique()

array(['Manufacturing', 'Real Estate and Rental and Leasing Services',
       nan, 'Construction', 'Retail Trade', 'Wholesale Trade',
       'Administrative and Support Services',
       'Private Education and Health Services',
       'Professional, Scientific, and Technical Services',
       'Food Services', 'Insurance', 'Certain Services',
       'Arts, Entertainment, and Recreation', 'Financial Services',
       'Information', 'Multiple', 'Transportation and Warehousing',
       'Accommodations', 'Utilities'], dtype=object)

In [56]:
# create a subset with only food service data from NAICS code
index_names1=df_bus[df_bus['NAICS_Code_Description'] != 'Food Services'].index
df_bust=df_bus.drop(index_names1)

In [57]:
df_bust['Business_Start_Date']=pd.to_datetime(df_bust['Business_Start_Date'])
df_bust['Business_End_Date']=pd.to_datetime(df_bust['Business_End_Date'])

In [58]:
df_bus_closed=df_bust.dropna(subset=['Business_End_Date'])

In [59]:
df_bus_o=df_bust.groupby(['Neighborhoods_-_Analysis_Boundaries', pd.Grouper(key='Business_Start_Date', freq='M')]).count()
df_bus_c=df_bus_closed.groupby(['Neighborhoods_-_Analysis_Boundaries', pd.Grouper(key='Business_End_Date', freq='M')]).count()

In [60]:
df_bus_o.reset_index(inplace=True)
df_bus_c.reset_index(inplace=True)

In [61]:
df_bus_o.drop(['Business_Account_Number', 'Ownership_Name', 'DBA_Name',
       'Street_Address', 'City', 'State', 'Source_Zipcode',
       'Business_End_Date', 'Location_Start_Date', 'Location_End_Date',
       'Mail_Address', 'Mail_City', 'Mail_Zipcode', 'Mail_State', 'NAICS_Code',
       'NAICS_Code_Description', 'Parking_Tax', 'Transient_Occupancy_Tax',
       'LIC_Code', 'LIC_Code_Description', 'Supervisor_District',
       'Business_Corridor', 'Business_Location', 'SF_Find_Neighborhoods',
       'Current_Police_Districts', 'Current_Supervisor_Districts',
       'Analysis_Neighborhoods', ':@computed_region_bh8s_q3mv',
       ':@computed_region_yftq_j783', ':@computed_region_rxqg_mtj9',
       ':@computed_region_jx4q_fizf', ':@computed_region_fyvs_ahh9',
       ':@computed_region_p5aj_wyqh'], axis=1, inplace=True)
df_bus_c.drop(['Business_Account_Number', 'Ownership_Name', 'DBA_Name',
       'Street_Address', 'City', 'State', 'Source_Zipcode',
       'Business_Start_Date', 'Location_Start_Date', 'Location_End_Date',
       'Mail_Address', 'Mail_City', 'Mail_Zipcode', 'Mail_State', 'NAICS_Code',
       'NAICS_Code_Description', 'Parking_Tax', 'Transient_Occupancy_Tax',
       'LIC_Code', 'LIC_Code_Description', 'Supervisor_District',
       'Business_Corridor', 'Business_Location', 'SF_Find_Neighborhoods',
       'Current_Police_Districts', 'Current_Supervisor_Districts',
       'Analysis_Neighborhoods', ':@computed_region_bh8s_q3mv',
       ':@computed_region_yftq_j783', ':@computed_region_rxqg_mtj9',
       ':@computed_region_jx4q_fizf', ':@computed_region_fyvs_ahh9',
       ':@computed_region_p5aj_wyqh'], axis=1, inplace=True)

In [62]:
df_bus_o.rename(columns={'Neighborhoods_-_Analysis_Boundaries': 'Neighborhood', 'Business_Start_Date': 'Date', 'Location_Id': 'Start_Count'}, inplace=True)
df_bus_c.rename(columns={'Neighborhoods_-_Analysis_Boundaries': 'Neighborhood', 'Business_End_Date': 'Date', 'Location_Id': 'End_Count'}, inplace=True)

In [63]:
df_bus_o. head()

,Neighborhood,Date,Start_Count
0,Bayview Hunters Point,1968-10-31,6
1,Bayview Hunters Point,1981-01-31,1
2,Bayview Hunters Point,1984-01-31,1
3,Bayview Hunters Point,1985-04-30,1
4,Bayview Hunters Point,1988-04-30,1


In [64]:
df_bus_o.describe()

,Start_Count
count,3379.000000
mean,1.874815
std,1.936381
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,58.000000


In [65]:
df_bus_c.head()

,Neighborhood,Date,End_Count
0,Bayview Hunters Point,2014-03-31,1
1,Bayview Hunters Point,2015-03-31,1
2,Bayview Hunters Point,2015-12-31,1
3,Bayview Hunters Point,2016-07-31,1
4,Bayview Hunters Point,2016-10-31,1


In [66]:
df_bus_c.describe()

,End_Count
count,663.000000
mean,1.805430
std,1.299489
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,12.000000


In [67]:
df_bus_o.shape

(3379, 3)

In [68]:
df_bus_c.shape

(663, 3)

# Mergin tables

In [69]:
df1=pd.merge(df_i1, df_bus_o[['Date', 'Start_Count', 'Neighborhood']], on=('Neighborhood', 'Date'), how='outer')

In [70]:
df2=pd.merge(df1, df_bus_c[['Neighborhood', 'Date', 'End_Count']], on=('Neighborhood', 'Date'), how='outer')

In [71]:
df3=pd.merge(df2, sf_rent[['Neighborhood', 'Avg_Rent']], on='Neighborhood', how='left')

In [72]:
df4=pd.merge(df3, sf_stats[['Neighborhood', 'Venue']], on='Neighborhood', how='left')

In [73]:
df1.fillna(0, inplace=True)
df2.fillna(0, inplace=True)
df3.fillna(0, inplace=True)
df4.fillna(0, inplace=True)

In [74]:
sc = StandardScaler()

In [75]:
df4_sc=df4

In [76]:
df4_sc[['Incidents', 'Start_Count', 'End_Count', 'Avg_Rent']] = sc.fit_transform(df4_sc[['Incidents', 'Start_Count', 'End_Count', 'Avg_Rent']])

In [77]:
df4_sc.head()

,Neighborhood,Date,Incidents,Start_Count,End_Count,Avg_Rent,Venue
0,Bayview Hunters Point,2018-01-31,4.172054,0.715446,-0.356093,0.633306,0.5
1,Bayview Hunters Point,2018-02-28,3.706739,0.192662,-0.356093,0.633306,0.5
2,Bayview Hunters Point,2018-03-31,3.392651,-0.852908,3.109345,0.633306,0.5
3,Bayview Hunters Point,2018-04-30,3.811435,0.192662,0.799053,0.633306,0.5
4,Bayview Hunters Point,2018-05-31,3.764903,0.192662,4.264491,0.633306,0.5


In [78]:
df4_sc=df4_sc.groupby('Neighborhood').mean()

In [79]:
df4_sc.reset_index(inplace=True)

In [80]:
df4_sc_clustering = df4_sc.drop('Neighborhood', 1)

In [81]:
KM=KMeans(n_clusters=4, random_state=0).fit_predict(df4_sc_clustering)

In [82]:
df4_sc.insert(0, 'Cluster_Labels', KM)

In [83]:
index_cluster_0=df4_sc[df4_sc['Cluster_Labels'] != 0].index
df4_sc_0=df4_sc.drop(index_cluster_0)
index_cluster_1=df4_sc[df4_sc['Cluster_Labels'] != 1].index
df4_sc_1=df4_sc.drop(index_cluster_1)
index_cluster_2=df4_sc[df4_sc['Cluster_Labels'] != 2].index
df4_sc_2=df4_sc.drop(index_cluster_2)
index_cluster_3=df4_sc[df4_sc['Cluster_Labels'] != 3].index
df4_sc_3=df4_sc.drop(index_cluster_3)

In [84]:
df4_sc_0.head()

,Cluster_Labels,Neighborhood,Incidents,Start_Count,End_Count,Avg_Rent,Venue
0,0,Bayview Hunters Point,0.205336,0.139978,0.020001,0.633306,0.500000
1,0,Bernal Heights,-0.120981,-0.261934,0.015563,0.310298,0.649123
2,0,Castro/Upper Market,0.007386,-0.100278,-0.007558,0.766309,0.610000
3,0,Chinatown,-0.151918,-0.031828,0.085580,0.646606,0.830000
8,0,Haight Ashbury,-0.089924,-0.245633,-0.122730,0.646606,0.473684


In [85]:
df4_sc_0.describe()

,Cluster_Labels,Incidents,Start_Count,End_Count,Avg_Rent,Venue
count,19.0,19.000000,19.000000,19.000000,19.000000,19.000000
mean,0.0,-0.022838,-0.096559,-0.009551,0.531470,0.604133
std,0.0,0.148939,0.188676,0.079884,0.213189,0.113372
min,0.0,-0.166313,-0.322080,-0.139503,0.286864,0.454545
25%,0.0,-0.115929,-0.253783,-0.063059,0.322964,0.493902
50%,0.0,-0.080890,-0.100278,-0.007558,0.520570,0.610000
75%,0.0,0.010439,-0.019251,0.022712,0.646606,0.665000
max,0.0,0.329599,0.416712,0.187505,0.936680,0.830000


In [86]:
df4_sc_1.head()

,Cluster_Labels,Neighborhood,Incidents,Start_Count,End_Count,Avg_Rent,Venue
13,1,Lakeshore,-0.116448,-0.162564,-0.104331,-1.553017,0.666667
14,1,Lincoln Park,-0.213196,-0.822156,-0.356093,-1.553017,0.000000
15,1,Lone Mountain/USF,-0.079044,-0.193395,-0.142835,-1.553017,0.516129
17,1,McLaren Park,-0.214992,-0.820234,-0.356093,-1.553017,0.000000
23,1,Oceanview/Merced/Ingleside,0.005784,-0.393877,0.010173,-1.553017,0.647059


In [87]:
df4_sc_1.describe()

,Cluster_Labels,Incidents,Start_Count,End_Count,Avg_Rent,Venue
count,9.0,9.000000,9.000000,9.000000,9.000000e+00,9.000000
mean,1.0,-0.121203,-0.387330,-0.145032,-1.553017e+00,0.415429
std,0.0,0.075266,0.356549,0.174470,1.156445e-15,0.267300
min,1.0,-0.214992,-0.852908,-0.356093,-1.553017e+00,0.000000
25%,1.0,-0.200814,-0.820234,-0.356093,-1.553017e+00,0.250000
50%,1.0,-0.111750,-0.238406,-0.104331,-1.553017e+00,0.516129
75%,1.0,-0.079044,-0.162564,-0.032302,-1.553017e+00,0.622642
max,1.0,0.005784,0.086107,0.114794,-1.553017e+00,0.666667


In [88]:
df4_sc_2.head()

,Cluster_Labels,Neighborhood,Incidents,Start_Count,End_Count,Avg_Rent,Venue
5,2,Financial District/South Beach,0.186414,1.151491,0.177450,-1.553017,0.66
18,2,Mission,0.269601,0.632429,0.194212,-1.553017,0.52


In [89]:
df4_sc_2.describe()

,Cluster_Labels,Incidents,Start_Count,End_Count,Avg_Rent,Venue
count,2.0,2.000000,2.000000,2.000000,2.000000e+00,2.000000
mean,2.0,0.228007,0.891960,0.185831,-1.553017e+00,0.590000
std,0.0,0.058822,0.367032,0.011853,6.280370e-16,0.098995
min,2.0,0.186414,0.632429,0.177450,-1.553017e+00,0.520000
25%,2.0,0.207210,0.762194,0.181640,-1.553017e+00,0.555000
50%,2.0,0.228007,0.891960,0.185831,-1.553017e+00,0.590000
75%,2.0,0.248804,1.021725,0.190022,-1.553017e+00,0.625000
max,2.0,0.269601,1.151491,0.194212,-1.553017e+00,0.660000


In [90]:
df4_sc_3.head()

,Cluster_Labels,Neighborhood,Incidents,Start_Count,End_Count,Avg_Rent,Venue
4,3,Excelsior,-0.058409,-0.266369,-0.102525,0.844844,0.333333
6,3,Glen Park,-0.161696,-0.500596,-0.155198,0.561737,0.000000
7,3,Golden Gate Park,0.256007,-0.710330,-0.303587,0.766309,0.244898
19,3,Mission Bay,-0.050257,0.041495,-0.049910,1.279322,0.560976
27,3,Portola,-0.038570,-0.426145,-0.143923,1.066517,0.738095


In [91]:
df4_sc_3.head()

,Cluster_Labels,Neighborhood,Incidents,Start_Count,End_Count,Avg_Rent,Venue
4,3,Excelsior,-0.058409,-0.266369,-0.102525,0.844844,0.333333
6,3,Glen Park,-0.161696,-0.500596,-0.155198,0.561737,0.000000
7,3,Golden Gate Park,0.256007,-0.710330,-0.303587,0.766309,0.244898
19,3,Mission Bay,-0.050257,0.041495,-0.049910,1.279322,0.560976
27,3,Portola,-0.038570,-0.426145,-0.143923,1.066517,0.738095


In [92]:
df4_sc_3.describe()

,Cluster_Labels,Incidents,Start_Count,End_Count,Avg_Rent,Venue
count,11.0,11.000000,11.000000,11.000000,11.000000,11.000000
mean,3.0,-0.055149,-0.438437,-0.173273,0.890733,0.254396
std,0.0,0.128285,0.232564,0.078107,0.373026,0.273123
min,3.0,-0.165899,-0.710330,-0.303587,0.103826,0.000000
25%,3.0,-0.142590,-0.627861,-0.228890,0.779293,0.000000
50%,3.0,-0.092091,-0.471711,-0.155198,0.844844,0.244898
75%,3.0,-0.044414,-0.344374,-0.123224,1.066517,0.460526
max,3.0,0.256007,0.041495,-0.049910,1.538362,0.738095
